In [1]:
"""Code to format info on movies in a csv file for publication."""

# libraries
import csv
import pandas as pd
import datetime

In [2]:
# read in csv file and prepare for work
filename = "2781_Fandango_data_Movie_sample_data.csv"
fieldnames = ['Location', 'Theatre', 'Movie', 'Movie_Rating', 'Show_Times', 'Movie_Format', 'Synopsis', 'Date', 'Searched_City']
#rows = []   

In [3]:
# Creats list of dictionaries for each row. This is working.

listings = []
scrape_file = '2781_Fandango_data_Movie_full.csv'
with open(scrape_file, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        layout_dct = {'addy': row[0], 'theater': row[1], 'title': row[2], 'mpaa': row[3], 'stars': row[4], 'times': row[5], 'format': row[6], 'synopsis': row[7], 'date': row[8], 'city': row[9]}
        listings.append(layout_dct)
        
del listings[0] #remove header row
        

In [4]:
print(listings[0])

{'addy': '2021 Silver Springs Blvd, Ocala, FL 34471', 'theater': 'Ocala Center 6', 'title': 'Ralph Breaks the Internet', 'mpaa': 'PG', 'stars': '4.5', 'times': '1:00p | 4:00p | 7:00p', 'format': 'Standard', 'synopsis': 'Ralph and Vanellope embark on an adventure inside the internet to find a spare part to fix a video game.', 'date': '2018-11-29', 'city': 'Ocala, FL'}


In [5]:
# CONVERTS datetime object into day of the week
def get_big_timestamp(date_object=None):
    if not date_object:
        date_object = datetime.datetime.now()
    stamp = ""
    stamp += datetime.datetime.strftime(date_object, "%A")
    return(stamp)

In [48]:
df = pd.DataFrame(listings)
times = (df.times.str.split('|', expand=True).rename(columns=lambda x: f"time_{x+1}"))
times = times.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace('p', ' p.m.') if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace('a', ' a.m.') if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace(':00', '') if x.dtype == "object" else x)
df = df.drop('times', axis=1)
df = pd.concat([df, times], axis=1, join_axes=[df.index])
df

,addy,city,date,format,mpaa,stars,synopsis,theater,title,time_1,time_2,time_3,time_4,time_5,time_6,time_7,time_8
0,"2021 Silver Springs Blvd, Ocala, FL 34471","Ocala, FL",2018-11-29,Standard,PG,4.5,Ralph and Vanellope embark on an adventure ins...,Ocala Center 6,Ralph Breaks the Internet,1 p.m.,4 p.m.,7 p.m.,None,None,None,None,None
1,"3702 Newberry Road, Gainesville, FL 32607","Gainesville, FL",2018-11-29,Standard,PG-13,5,"Singer Freddie Mercury, guitarist Brian May, d...",Regal Royal Park Stadium 16,Bohemian Rhapsody,12:15 p.m.,3:20 p.m.,6:20 p.m.,9:30 p.m.,None,None,None,None
2,"3702 Newberry Road, Gainesville, FL 32607","High Springs, FL",2018-11-29,Standard,PG,4.5,Academy Award® nominee Benedict Cumberbatch le...,Regal Royal Park Stadium 16,Dr. Seuss' The Grinch (2018),12:05 p.m.,2:10 p.m.,4:25 p.m.,6:50 p.m.,9:10 p.m.,None,None,None
3,"2801 SW 27th Avenue, Ocala, FL 34474","Belleview, FL",2018-11-29,Standard,PG-13,5,"Singer Freddie Mercury, guitarist Brian May, d...",Regal Hollywood Stadium 16 & IMAX - Ocala,Bohemian Rhapsody,12:25 p.m.,3:55 p.m.,10:35 p.m.,None,None,None,None,None
4,"2720 Brownwood Blvd., The Villages, FL 32163","The Villages, FL",2018-11-29,Standard,PG-13,4.5,Reporter Eddie Brock develops superpowers afte...,Barnstorm Theater,Venom (2018),1 p.m.,4 p.m.,9:40 p.m.,None,None,None,None,None
5,"3101 SW 35th Blvd, Gainesville, FL 32608","Gainesville, FL",2018-11-29,Standard,,5,,Regal Butler Town Center 14,2.0,12 p.m.,None,None,None,None,None,None,None
6,"4901 SW 31st Place, Gainesville, FL 32608","High Springs, FL",2018-11-29,Standard,PG,4.5,Academy Award® nominee Benedict Cumberbatch le...,Celebration Pointe 10,Dr. Seuss' The Grinch (2018),1:15 p.m.,4:30 p.m.,7:10 p.m.,9:30 p.m.,None,None,None,None
7,"4901 SW 31st Place, Gainesville, FL 32608","Gainesville, FL",2018-11-29,Standard,PG-13,4.5,In an effort to thwart Grindelwald's plans of ...,Celebration Pointe 10,Fantastic Beasts: The Crimes of Grindelwald,12 p.m.,3:15 p.m.,7 p.m.,1 p.m.,4:15 p.m.,7:30 p.m.,None,None
8,"3101 SW 35th Blvd, Gainesville, FL 32608","Gainesville, FL",2018-11-29,Standard,PG,4.5,Academy Award® nominee Benedict Cumberbatch le...,Regal Butler Town Center 14,Dr. Seuss' The Grinch (2018),12:05 p.m.,2:30 p.m.,4:50 p.m.,10:45 p.m.,7:15 p.m.,None,None,None
9,"101 W. Call St., Starke, FL 32091","Starke, FL",2018-11-29,Standard,PG,4.5,Ralph and Vanellope embark on an adventure ins...,Florida Twin,Ralph Breaks the Internet,7:30 p.m.,None,None,None,None,None,None,None


In [ ]:
df = pd.DataFrame(listings)
df['streetaddy'] = df.addy.str.split(',').str[0]
df.streetaddy = df.streetaddy.str.replace('Blvd', 'Blvd.')
df.streetaddy = df.streetaddy.str.replace('Blvd..', 'Blvd.')
df.streetaddy = df.streetaddy.str.replace('Avenue', 'Ave')
df.streetaddy = df.streetaddy.str.replace('Ave', 'Ave.')
df.streetaddy = df.streetaddy.str.replace('Ave..', 'Ave.')
df.times = df.times.str.replace('|', ',')
df.times = df.times.str.replace(':00p', ' p.m.')
#df.times = df.times.str.replace('0p', '0 p.m.')
#df.times = df.times.str.replace('5p', '5 p.m.')
#df.times = df.times.str.replace('. , ', '., ')

df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['movieday'] = df['date'].dt.weekday_name
df.times


In [ ]:
(df.times.str.split(',', expand=True).rename(columns=lambda x: f"string_{x+1}"))

In [ ]:
class Showing:
    def __init__(self, )
    
    
    
    
    
    class Cinema:
    def __init__(self, theater, street, city, market):
        self.theater = theater
        self.street = street
        self.city = city
        self.market = market
        
    def __str__(self):
        return f"{self.theater}, {self.stree}, {self.city}"
    
    def __repr__(self):
        return f"{self.theater}, {self.stree}, {self.city}"

class Movie:
    def __init__(self, title, mpaa, stars, synopsis):
        self.title = title
        self.mpaa = mpaa
        self.stars = stars
        self.synopsis = synopsis
        
    def __str__(sekf):
        return f"{title}, {mpaa}, {stars}, {synopsis}"
    
    def __str__(sekf):
        return f"{title}, {mpaa}, {stars}, {synopsis}"
    
class Showing:
    